In [ ]:
!pip install --upgrade numpy --target ./python
!pip install --upgrade numexpr --target ./python

In [1]:
import sys
sys.path.append(r"./python")

import os
import json
from model import *

#根据时间情况修改index和language值
index =  ""
embedding_endpoint_name = "cohere.embed-multilingual-v3"

embedding_type = 'bedrock' if embedding_endpoint_name.find('titan') or embedding_endpoint_name.find('cohere') else 'sagemaker'
embeddings = init_embeddings_bedrock(embedding_endpoint_name)

In [ ]:
import sys
sys.path.append(r"./python")

from tqdm import tqdm
import fitz
from PIL import Image
import numpy as np
import base64
from opensearch_multimodel_dataload import add_multimodel_documents
import re

model_name = "anthropic.claude-3-5-sonnet-20240620-v1:0"
# model_name = "anthropic.claude-3-sonnet-20240229-v1:0"
llm = init_model_bedrock(model_name)
text_max_length = 2000

def is_json(myjson):
    try:
        json.loads(myjson)
    except ValueError as e:
        return False
    return True

prompt = """
You are a document manager at an insurance company and your task is to extract useful information from document images.
<instructions>
1.Output the document in markdown format, and keep the rows and columns aligned for the table.
2.To retain all headers and table content, they cannot be omitted
3.No preface, just output the document content directly.
</instructions>
"""

files_path = '../docs/xxxxx/'
# os.mkdir('images/')
files = os.listdir(files_path)
for file in files:
    file_path = files_path + file
    print(file_path)
    # fname = file.split('/')[-1].split('.')[0]
    # print(fname)
    # os.mkdir('images/'+fname)

    doc = fitz.open(file_path)
    previous_page_content = ''
    
    texts = []
    metadatas = []
    images = []
    
    for i in tqdm(range(doc.page_count)):

        page = doc.load_page(i)
        pix = page.get_pixmap(dpi=100)
        
        # img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        # img_name = fname + '-' + str(i) + '.jpg'
        # img.save('images/'+fname + '/' + img_name, "JPEG")
        
        imgb64 = base64.b64encode(pix.tobytes()).decode("utf-8")
        model_kwargs = {'image': imgb64,'max_tokens':2048}
        llm.model_kwargs = model_kwargs
        new_prompt = prompt.format(context=previous_page_content)
        response = llm(prompt=new_prompt)
        response = response.strip()
        previous_page_content = response
        
        print('response:',response)

        response_set = set()
        response_list = response.split('\n')
        
        header = []
        for sentence in response_list:
            sentence = sentence.strip()
            data_list = re.split(r"\||<br>|/", sentence)
            for data in data_list:
                data = sentence.strip()
                if len(data) > 0:
                    response_set.add(data)
            
            if sentence.find('|') >=0:
                if len(header) == 0:
                    header = sentence.split('|')
                else:
                    content = sentence.replace('-','').replace(':','').replace('|','')
                    if len(content) > 0:
                        content = sentence.split('|')
                        line_str = ''
                        for i in range(len(header)):
                            if i < len(content) and len(str(header[i]).strip()) > 0 and len(str(content[i]).strip()) > 0:
                                line_str += (str(header[i]).strip() + ':' + str(content[i]).strip()+ ',')
                            elif i < len(content) and len(str(content[i]).strip()) > 0:
                                line_str += (str(content[i]).strip()+ ',')
                            else:
                                line_str += (str(header[i]).strip()+ ',')
                        response_set.add(line_str[:-1])

            elif sentence.find('|') < 0 and len(header) > 0:
                header = []
                
        for text in response_set:
            text = text.strip()
            print('text:',text)
            print('--------------')
            texts.append(str(response))
            images.append(imgb64)
            metadata = {}
            metadata['sentence'] = text[:text_max_length] if len(text) > text_max_length else text
            metadata['sources'] = file.split('/')[-1]
            metadata['page'] = i
            metadatas.append(metadata)
            
        if len(texts) > 0:
            if embedding_type == 'bedrock':
                text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas])
            else:
                text_embeddings = embeddings.embed_documents([metadata['sentence'] for metadata in metadatas],chunk_size=10)

            print('texts len:',len(texts))
            print('metadatas len:',len(metadatas))
            print('embeddings len:',len(text_embeddings))
            print('images len:',len(images))
            print('begin to save in vectore store')

            add_multimodel_documents(
                index,
                texts=texts,
                embeddings=text_embeddings,
                metadatas=metadatas,
                images=images
            )
        print('finish save in vectore store:',index)
        texts = []
        metadatas = []
        images = []

